In [ ]:
#To install the required packages uncomment the following line
%pip install seaborn mplhep

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mplhep
mplhep.style.use(mplhep.style.ROOT)
plt.rcParams.update({'font.size': 16,
                        'grid.linestyle': '--',
                        'axes.grid': True,
                        'figure.autolayout': True,
                        'figure.figsize': [14,6]
                        })

In [ ]:
url = "https://docs.google.com/spreadsheets/d/1euDfLLXG5vzFyvdg0-5Btzs14547jU6ah6n5oljttTo/export?format=xlsx"
df_c = pd.read_excel(url, sheet_name="Calibration_C")
df_m = pd.read_excel(url, sheet_name="Calibration_M")
#df = pd.read_excel(url, sheet_name="Calibration_M")
df_c.head()  # this is to check that everything is fine

In [ ]:
def create_relative_diff(df: pd.DataFrame):
    columns_name = df.drop(columns=['RUN','DAY', 'ENDPOINT', 'MODULE']).columns
    for col in columns_name:
        col:str
        if col == "CHANNEL" or col.endswith('_diff'):
            continue
        df[f'{col}_diff'] =  (df[col] - df.groupby('CHANNEL')[col].transform('mean')) / df.groupby('CHANNEL')[col].transform('mean')
        # df[f'{col}_mean'] =  df.groupby('CHANNEL')[col].transform('mean')

create_relative_diff(df_c)
create_relative_diff(df_m)

In [ ]:
def plot_values_vs_time(df:pd.DataFrame, saveit=False, plotdiff=False):
    df['DAY'] = pd.to_datetime(df['DAY'])
    
    module_index = df.columns.get_loc("MODULE")
    columns_to_plot = df.columns[module_index+1:]
    
    tmpmname:str = df.iloc[0]['MODULE']
    detector_type = 'cathode'
    if tmpmname.startswith("M"):
        detector_type = 'membrane'
    
    for col in columns_to_plot:
        col:str
        if not plotdiff and col.endswith('_diff'):
            continue
        if plotdiff and not col.endswith('_diff'):
            continue
        plt.figure(figsize=(13, 6))
        sns.lineplot(
        #sns.scatterplot(
            data=df,
            x='DAY',
            y=col,
            hue='MODULE',   
            style='MODULE',    # different marker shapes
            markers=True,
            #marker='o',
            dashes=False,
            linewidth=1.0,   
            markersize=8     
        )
    
        y_min, y_max = df[col].min(), df[col].max()
        margin = (y_max - y_min) * 0.5
        if not plotdiff:
            plt.ylim(y_min - margin, y_max + margin)
        else:
            plt.ylim(-0.3,0.3)
            
        
        plt.legend(title="MODULE", bbox_to_anchor=(1.02, 1), loc='upper left')
        # plt.xticks(rotation=45)
        plt.xlabel("Date")
        plt.tight_layout()
        if saveit:
            plt.savefig(f'{detector_type}_{col}.png', dpi=200) 


In [ ]:
plot_values_vs_time(df_c, saveit=False, plotdiff=False)
plot_values_vs_time(df_m, saveit=False, plotdiff=False)

In [ ]:
plot_values_vs_time(df_c, saveit=False, plotdiff=True)
plot_values_vs_time(df_m, saveit=False, plotdiff=True)